In [1]:
import copy

In [2]:
def eval_stato(stato):
        
    lunghezza_percorso = 0
    for i in range(len(stato)):
        l = connections[stato[i]]
        if i < (len(stato) - 1):
            nodo_succ = stato[i + 1]
        else:
            nodo_succ = stato[0]
        for (successore, distanza) in l:
            if successore == nodo_succ:
                lunghezza_percorso += distanza
                break
    return(lunghezza_percorso)

In [3]:
def generazione_successori(stato): 
    """"
    genera la lista ordinata di successori di uno stato
    """""
        
    lista = []
    t = len(stato)
    
    for i in range(0, t-1):
        for j in range(i+1, t):
            buffer = copy.deepcopy(stato)
            temp = buffer[i]
            buffer[i] = buffer[j]
            buffer[j] = temp
            eval_successore = eval_stato(buffer)
            lista.append((buffer, eval_successore, (stato[i], stato[j])))  

    lista.sort(key=lambda x: x[1])  # ordiniamo i successori in base alla loro valutazione eval_stato
    return(lista)
        

In [4]:
def tabu_test(sequenza, tabu_list):    # è True se una mossa NON è presente nella tabu_list
    
    a, b = sequenza[2]
    if ((a, b) in tabu_list or (b, a) in tabu_list):
        assente = False
    else:
        assente = True
    return(assente)

In [5]:
def tabu_search(stato_iniziale, tabu_tenure, connections):
           
    # impostazione stato iniziale
    current = stato_iniziale
    eval_current = eval_stato(current)
    
    # inizializzazione best
    best = stato_iniziale
    eval_best = eval_current
        
    tabu_list = {}
    neighbours = []
        
    cont = 0
    
    # while not criterio_terminazione():
    while cont < 10:
            
        # generazione successori (stato, eval_stato, mossa) e ordinamento su eval_stato
        lista_successori = generazione_successori(current)           
            
        # selezione successori non tabu
        neighbours = list(filter(lambda n: tabu_test(n, tabu_list), lista_successori))
            
        next_state = neighbours[0][0]         # selezione del migliore dei successori
        eval_next_state = neighbours[0][1]
        delta = eval_best - eval_next_state
        if delta > 0:
            best = next_state                 # aggiornamento di best
            eval_best = eval_next_state       
        current = next_state
        eval_current = eval_next_state
            
        # decremento del tabu_tenure
        for mossa in tabu_list:
            tabu_list[mossa] = tabu_list[mossa] - 1
                
        # eliminazione elementi con tenure uguale a zero 
        tabu_list = {k: tabu_list[k] for k in tabu_list if tabu_list[k]!=0}   
  
        # inserimento della mossa di next nella tabu_list
        mossa_next = neighbours[0][2]
        tabu_list[mossa_next] = tabu_tenure
            
        cont += 1
                
    return(best + [best[0]], eval_best)

In [6]:
# connessioni tra città con relative distanze

connections = {}

connections['A'] = [['B', 4], ['C', 2], ['D',  1], ['E',  2]]
connections['B'] = [['A', 4], ['C', 1], ['D', 2], ['E', 3]]
connections['C'] = [['A', 2], ['B', 1], ['D', 3], ['E',  2]]
connections['D'] = [['A', 1], ['B', 2], ['C',  3], ['E',  2]]
connections['E'] = [['A', 2], ['B',  3], ['C',  2], ['D',  2]]


In [7]:
stato = ['A', 'B', 'C', 'D', 'E']

In [8]:
tabu_search(stato, 3, connections)

(['C', 'B', 'D', 'A', 'E', 'C'], 8)